In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("earthquake_1995-2023.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   title      1000 non-null   object 
 1   magnitude  1000 non-null   float64
 2   date_time  1000 non-null   object 
 3   cdi        1000 non-null   int64  
 4   mmi        1000 non-null   int64  
 5   alert      449 non-null    object 
 6   tsunami    1000 non-null   int64  
 7   sig        1000 non-null   int64  
 8   net        1000 non-null   object 
 9   nst        1000 non-null   int64  
 10  dmin       1000 non-null   float64
 11  gap        1000 non-null   float64
 12  magType    1000 non-null   object 
 13  depth      1000 non-null   float64
 14  latitude   1000 non-null   float64
 15  longitude  1000 non-null   float64
 16  location   994 non-null    object 
 17  continent  284 non-null    object 
 18  country    651 non-null    object 
dtypes: float64(6), int64(5), object(8)
memory usage: 

In [4]:
df.head()

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
0,"M 6.5 - 42 km W of Sola, Vanuatu",6.5,16-08-2023 12:47,7,4,green,0,657,us,114,7.177000,25.0,mww,192.955,-13.8814,167.1580,"Sola, Vanuatu",NaN,Vanuatu
1,"M 6.5 - 43 km S of Intipucá, El Salvador",6.5,19-07-2023 00:22,8,6,yellow,0,775,us,92,0.679000,40.0,mww,69.727,12.8140,-88.1265,"Intipucá, El Salvador",NaN,NaN
2,"M 6.6 - 25 km ESE of Loncopué, Argentina",6.6,17-07-2023 03:05,7,5,green,0,899,us,70,1.634000,28.0,mww,171.371,-38.1911,-70.3731,"Loncopué, Argentina",South America,Argentina
3,"M 7.2 - 98 km S of Sand Point, Alaska",7.2,16-07-2023 06:48,6,6,green,1,860,us,173,0.907000,36.0,mww,32.571,54.3844,-160.6990,"Sand Point, Alaska",NaN,NaN
4,M 7.3 - Alaska Peninsula,7.3,16-07-2023 06:48,0,5,NaN,1,820,at,79,0.879451,172.8,Mi,21.000,54.4900,-160.7960,Alaska Peninsula,NaN,NaN


In [5]:
df.isna().sum()

title          0
magnitude      0
date_time      0
cdi            0
mmi            0
alert        551
tsunami        0
sig            0
net            0
nst            0
dmin           0
gap            0
magType        0
depth          0
latitude       0
longitude      0
location       6
continent    716
country      349
dtype: int64

##  Preprocessing

### Fixing null values for Null in location


In [6]:
#Locating null values
df_loc_na = df[df['location'].isna()]
df_loc_na

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
30,M 6.8 -,6.8,20-01-2023 22:09,4,3,green,0,718,us,60,5.129,61.0,mww,610.671,-26.7209,-63.0394,NaN,South America,Argentina
37,M 7.0 -,7.0,12-11-2022 07:09,3,3,green,1,755,us,147,3.125,18.0,mww,579.000,-20.0508,-178.3460,NaN,Oceania,Fiji
39,M 6.6 -,6.6,09-11-2022 10:14,0,2,green,1,670,us,131,4.998,27.0,mww,624.464,-25.5948,178.2780,NaN,NaN,NaN
54,M 6.9 -,6.9,19-05-2022 10:13,2,5,green,1,733,us,127,0.371,45.0,mww,10.000,-54.1325,159.0270,NaN,NaN,NaN
103,M 6.9 -,6.9,01-05-2021 01:27,7,6,green,1,919,us,0,2.619,35.0,mww,43.000,38.2296,141.6650,NaN,Asia,Japan
281,M 6.9 -,6.9,28-05-2016 05:38,3,3,green,1,733,us,0,5.485,19.0,mww,405.690,-21.9724,-178.2040,NaN,NaN,NaN


In [7]:
from geopy.geocoders import Nominatim
import time

In [8]:
# Using geopy library to replace na values in location using latitude and longitude column
def impute_locations(df, lat_col='latitude', lon_col='longitude', location_col='location'):
   
    df_copy = df.copy()
    # Set language preference to English
    geolocator = Nominatim(user_agent="my_agent")
    
    def get_location(row):
        if pd.isna(row[location_col]):
            try:
                coords = f"{row[lat_col]}, {row[lon_col]}"
                # Request data in English
                location = geolocator.reverse(coords, language='en')
                
                if location and location.raw.get('address'):
                    address = location.raw['address']
                    # Get English names for state/province and country
                    state = (
                        address.get('state_en') or 
                        address.get('state') or 
                        address.get('province_en') or 
                        address.get('province') or 
                        ''
                    )
                    country = address.get('country_en') or address.get('country', '')
                    return f"{state}, {country}".strip(', ')
                return None
            except Exception as e:
                print(f"Error getting location for coordinates {coords}: {e}")
                return None
            finally:
                time.sleep(1)
        return row[location_col]
    
    mask = df_copy[location_col].isna()
    if mask.any():
        print(f"Imputing {mask.sum()} missing locations...")
        df_copy[location_col] = df_copy.apply(get_location, axis=1)
    
    return df_copy


In [9]:
#Applying impute_location
df = impute_locations(df)
df.isna().sum()

Imputing 6 missing locations...


title          0
magnitude      0
date_time      0
cdi            0
mmi            0
alert        551
tsunami        0
sig            0
net            0
nst            0
dmin           0
gap            0
magType        0
depth          0
latitude       0
longitude      0
location       3
continent    716
country      349
dtype: int64

In [10]:
print(df.iloc[37])

title                M 7.0 - 
magnitude                 7.0
date_time    12-11-2022 07:09
cdi                         3
mmi                         3
alert                   green
tsunami                     1
sig                       755
net                        us
nst                       147
dmin                    3.125
gap                      18.0
magType                   mww
depth                   579.0
latitude             -20.0508
longitude            -178.346
location        Eastern, Fiji
continent             Oceania
country                  Fiji
Name: 37, dtype: object


In [11]:
df_loc_na = df[df['location'].isna()]
df_loc_na

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
39,M 6.6 -,6.6,09-11-2022 10:14,0,2,green,1,670,us,131,4.998,27.0,mww,624.464,-25.5948,178.278,None,NaN,NaN
54,M 6.9 -,6.9,19-05-2022 10:13,2,5,green,1,733,us,127,0.371,45.0,mww,10.000,-54.1325,159.027,None,NaN,NaN
281,M 6.9 -,6.9,28-05-2016 05:38,3,3,green,1,733,us,0,5.485,19.0,mww,405.690,-21.9724,-178.204,None,NaN,NaN


In [12]:
#Dropping null values in location column
df.dropna(subset = "location",axis = 0,inplace = True)

In [13]:
df.isna().sum()

title          0
magnitude      0
date_time      0
cdi            0
mmi            0
alert        551
tsunami        0
sig            0
net            0
nst            0
dmin           0
gap            0
magType        0
depth          0
latitude       0
longitude      0
location       0
continent    713
country      346
dtype: int64

In [14]:
df.head(30)

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
0,"M 6.5 - 42 km W of Sola, Vanuatu",6.5,16-08-2023 12:47,7,4,green,0,657,us,114,7.177000,25.0,mww,192.955,-13.8814,167.1580,"Sola, Vanuatu",NaN,Vanuatu
1,"M 6.5 - 43 km S of Intipucá, El Salvador",6.5,19-07-2023 00:22,8,6,yellow,0,775,us,92,0.679000,40.0,mww,69.727,12.8140,-88.1265,"Intipucá, El Salvador",NaN,NaN
2,"M 6.6 - 25 km ESE of Loncopué, Argentina",6.6,17-07-2023 03:05,7,5,green,0,899,us,70,1.634000,28.0,mww,171.371,-38.1911,-70.3731,"Loncopué, Argentina",South America,Argentina
3,"M 7.2 - 98 km S of Sand Point, Alaska",7.2,16-07-2023 06:48,6,6,green,1,860,us,173,0.907000,36.0,mww,32.571,54.3844,-160.6990,"Sand Point, Alaska",NaN,NaN
4,M 7.3 - Alaska Peninsula,7.3,16-07-2023 06:48,0,5,NaN,1,820,at,79,0.879451,172.8,Mi,21.000,54.4900,-160.7960,Alaska Peninsula,NaN,NaN
5,"M 6.6 - 277 km NNE of Codrington, Antigua and ...",6.6,10-07-2023 20:28,5,4,green,1,802,us,95,2.454000,37.0,mww,10.000,20.0196,-61.0955,"Codrington, Antigua and Barbuda",NaN,NaN
6,M 6.9 - Tonga,6.9,02-07-2023 10:27,4,4,green,1,741,us,136,1.179000,23.0,mww,229.000,-17.8530,-174.9370,Tonga,NaN,NaN
7,M 7.2 - south of the Fiji Islands,7.2,15-06-2023 18:06,8,6,green,1,804,us,85,2.590000,24.0,mww,167.404,-22.9824,-177.2080,the Fiji Islands,NaN,NaN
8,M 6.6 - Panama-Colombia border region,6.6,25-05-2023 03:05,6,6,green,1,733,us,50,2.163000,129.0,mww,10.000,8.8894,-77.1231,Panama-Colombia border region,NaN,Colombia
9,M 7.1 - southeast of the Loyalty Islands,7.1,20-05-2023 01:51,3,4,green,1,777,us,98,2.812000,56.0,mww,35.981,-23.0622,170.4560,the Loyalty Islands,NaN,NaN


### Fixing null values of continent column

In [15]:
df["continent"].isna().sum()

np.int64(713)

In [16]:
df["continent"].unique()

array([nan, 'South America', 'Asia', 'Oceania', 'North America', 'Europe',
       'Africa'], dtype=object)

In [17]:
# Function to get continent using latitude and longitude
def get_continent(lat, lon):
    """
    Determines the continent for a given latitude and longitude.

    Parameters:
    lat (float): Latitude
    lon (float): Longitude

    Returns:
    str: The name of the continent or 'Unknown' if not found.
    """
    try:
        # Use geopy to get the country
        geolocator = Nominatim(user_agent="continent_finder")
        location = geolocator.reverse((lat, lon), language='en')
        
        if location and 'country_code' in location.raw['address']:
            country_code = location.raw['address']['country_code'].upper()
            
            # Get the continent using pycountry_convert
            continent_code = pc.country_alpha2_to_continent_code(country_code)
            continents = {
                "AF": "Africa",
                "NA": "North America",
                "SA": "South America",
                "AS": "Asia",
                "EU": "Europe",
                "OC": "Oceania",
                "AN": "Antarctica"
            }
            return continents.get(continent_code, "Unknown")
        else:
            return "Unknown"
    except Exception as e:
        return f"Error: {e}"
    finally:
        time.sleep(1)  # Ensure the rate limit is respected

In [18]:
# Apply the function only to rows where continent is NaN
#df['continent'] = df.apply(
#    lambda row: get_continent(row['latitude'], row['longitude']) if pd.isna(row['continent']) else row['continent'], axis=1
#
#print(df.isna().sum())

### Fixing null values in country 

In [19]:
df["location"].unique()

array(['Sola, Vanuatu', 'Intipucá, El Salvador', 'Loncopué, Argentina',
       'Sand Point, Alaska', 'Alaska Peninsula',
       'Codrington, Antigua and Barbuda', 'Tonga', 'the Fiji Islands',
       'Panama-Colombia border region', 'the Loyalty Islands',
       'Hihifo, Tonga', 'Teluk Dalam, Indonesia',
       'Kermadec Islands, New Zealand', 'Tuban, Indonesia',
       'the Kamchatka Peninsula, Russia', 'New Guinea, Papua New Guinea',
       'San Antonio de los Cobres, Argentina', 'Jurm, Afghanistan',
       'Ecuador', 'Kermadec Islands region', 'Port-Olry, Vanuatu',
       'Kimbe, Papua New Guinea', 'Murghob, Tajikistan',
       'Ekinözü, Turkey', 'Nurdağı, Turkey', 'Central Turkey',
       'Santiago del Estero, Argentina', 'Tobelo, Indonesia',
       'Pulau Pulau Tanimbar, Indonesia', 'Vanuatu',
       'Malango, Solomon Islands', 'Bengkulu, Indonesia', 'Eastern, Fiji',
       'Neiafu, Tonga', 'Boca Chica, Panama', 'Aguililla, Mexico',
       'Yujing, Taiwan', 'Lugu, Taiwan', 'Isangel

In [20]:
# Function to extract values after comma in location column
def update_country_column(df, source_col, country_col):
    
    df[country_col] = df.apply(
        lambda row: row[source_col].split(',')[-1].strip() 
        if pd.isna(row[country_col]) and ',' in row[source_col] 
        else row[country_col], 
        axis=1
    )
    return df

In [21]:
# Using update_country_column
df = update_country_column(df,"location","country")
df.isna().sum()

title          0
magnitude      0
date_time      0
cdi            0
mmi            0
alert        551
tsunami        0
sig            0
net            0
nst            0
dmin           0
gap            0
magType        0
depth          0
latitude       0
longitude      0
location       0
continent    713
country       51
dtype: int64

In [22]:
df[df["country"].isna()]

,title,magnitude,date_time,cdi,mmi,alert,tsunami,sig,net,nst,dmin,gap,magType,depth,latitude,longitude,location,continent,country
4,M 7.3 - Alaska Peninsula,7.3,16-07-2023 06:48,0,5,NaN,1,820,at,79,0.879451,172.8,Mi,21.000,54.4900,-160.7960,Alaska Peninsula,NaN,NaN
6,M 6.9 - Tonga,6.9,02-07-2023 10:27,4,4,green,1,741,us,136,1.179000,23.0,mww,229.000,-17.8530,-174.9370,Tonga,NaN,NaN
7,M 7.2 - south of the Fiji Islands,7.2,15-06-2023 18:06,8,6,green,1,804,us,85,2.590000,24.0,mww,167.404,-22.9824,-177.2080,the Fiji Islands,NaN,NaN
9,M 7.1 - southeast of the Loyalty Islands,7.1,20-05-2023 01:51,3,4,green,1,777,us,98,2.812000,56.0,mww,35.981,-23.0622,170.4560,the Loyalty Islands,NaN,NaN
10,M 7.7 - southeast of the Loyalty Islands,7.7,19-05-2023 02:57,5,4,green,1,927,us,277,3.111000,15.0,mww,18.027,-23.2286,170.6940,the Loyalty Islands,NaN,NaN
12,M 6.6 - south of the Fiji Islands,6.6,28-04-2023 03:13,0,2,green,0,670,us,52,5.123000,50.0,mww,598.467,-25.2657,178.4240,the Fiji Islands,NaN,NaN
15,M 6.7 - south of the Fiji Islands,6.7,18-04-2023 04:31,0,3,green,1,691,us,166,4.719000,10.0,mww,595.854,-22.2844,179.3910,the Fiji Islands,NaN,NaN
22,M 7.0 - Kermadec Islands region,7.0,16-03-2023 00:56,8,5,green,1,756,us,116,1.789000,22.0,mww,22.065,-30.1147,-176.1120,Kermadec Islands region,NaN,NaN
40,M 7.0 - south of the Fiji Islands,7.0,09-11-2022 09:51,4,3,green,1,755,us,142,4.578000,26.0,mwb,660.000,-26.0442,178.3810,the Fiji Islands,NaN,NaN
41,M 6.8 - south of the Fiji Islands,6.8,09-11-2022 09:38,1,3,green,1,711,us,136,4.678000,22.0,mww,630.379,-25.9678,178.3630,the Fiji Islands,NaN,NaN


In [23]:
df.iloc[6]

title           M 6.9 - Tonga
magnitude                 6.9
date_time    02-07-2023 10:27
cdi                         4
mmi                         4
alert                   green
tsunami                     1
sig                       741
net                        us
nst                       136
dmin                    1.179
gap                      23.0
magType                   mww
depth                   229.0
latitude              -17.853
longitude            -174.937
location                Tonga
continent                 NaN
country                   NaN
Name: 6, dtype: object

In [24]:
df[df["country"].isna()]["location"].unique()

array(['Alaska Peninsula', 'Tonga', 'the Fiji Islands',
       'the Loyalty Islands', 'Kermadec Islands region',
       'the Kermadec Islands', 'Vanuatu region',
       'South Sandwich Islands region', 'Mauritius - Reunion region',
       'central Mid-Atlantic Ridge', 'Prince Edward Islands region',
       'Bouvet Island region', 'Fiji region', 'Micronesia region',
       'Okhotsk', 'northern Mid-Atlantic Ridge',
       'off the west coast of northern Sumatra',
       'Philippine Islands region', 'the Kuril Islands', 'Kuril Islands',
       'Macquarie Island', 'South Indian Ocean'], dtype=object)

In [25]:
uniq_location_country_na = list(df[df["country"].isna()]["location"].unique())

In [26]:
import nltk
uniq_location_words_country_na=[]
for i in uniq_location_country_na: 
    words_location = nltk.word_tokenize(str(i))
    for j in words_location: 
        uniq_location_words_country_na.append(j)

In [27]:
uniq_location_words_country_na

['Alaska',
 'Peninsula',
 'Tonga',
 'the',
 'Fiji',
 'Islands',
 'the',
 'Loyalty',
 'Islands',
 'Kermadec',
 'Islands',
 'region',
 'the',
 'Kermadec',
 'Islands',
 'Vanuatu',
 'region',
 'South',
 'Sandwich',
 'Islands',
 'region',
 'Mauritius',
 '-',
 'Reunion',
 'region',
 'central',
 'Mid-Atlantic',
 'Ridge',
 'Prince',
 'Edward',
 'Islands',
 'region',
 'Bouvet',
 'Island',
 'region',
 'Fiji',
 'region',
 'Micronesia',
 'region',
 'Okhotsk',
 'northern',
 'Mid-Atlantic',
 'Ridge',
 'off',
 'the',
 'west',
 'coast',
 'of',
 'northern',
 'Sumatra',
 'Philippine',
 'Islands',
 'region',
 'the',
 'Kuril',
 'Islands',
 'Kuril',
 'Islands',
 'Macquarie',
 'Island',
 'South',
 'Indian',
 'Ocean']

In [28]:
tag_elements = nltk.pos_tag(uniq_location_words_country_na)

In [29]:
tag_elements

[('Alaska', 'NNP'),
 ('Peninsula', 'NNP'),
 ('Tonga', 'NNP'),
 ('the', 'DT'),
 ('Fiji', 'NNP'),
 ('Islands', 'VBZ'),
 ('the', 'DT'),
 ('Loyalty', 'NNP'),
 ('Islands', 'NNP'),
 ('Kermadec', 'NNP'),
 ('Islands', 'NNP'),
 ('region', 'NN'),
 ('the', 'DT'),
 ('Kermadec', 'NNP'),
 ('Islands', 'NNP'),
 ('Vanuatu', 'NNP'),
 ('region', 'NN'),
 ('South', 'NNP'),
 ('Sandwich', 'NNP'),
 ('Islands', 'NNP'),
 ('region', 'NN'),
 ('Mauritius', 'NNP'),
 ('-', ':'),
 ('Reunion', 'NN'),
 ('region', 'NN'),
 ('central', 'JJ'),
 ('Mid-Atlantic', 'JJ'),
 ('Ridge', 'NNP'),
 ('Prince', 'NNP'),
 ('Edward', 'NNP'),
 ('Islands', 'NNP'),
 ('region', 'NN'),
 ('Bouvet', 'NNP'),
 ('Island', 'NNP'),
 ('region', 'NN'),
 ('Fiji', 'NNP'),
 ('region', 'NN'),
 ('Micronesia', 'NNP'),
 ('region', 'NN'),
 ('Okhotsk', 'NNP'),
 ('northern', 'JJ'),
 ('Mid-Atlantic', 'NNP'),
 ('Ridge', 'NNP'),
 ('off', 'IN'),
 ('the', 'DT'),
 ('west', 'JJS'),
 ('coast', 'NN'),
 ('of', 'IN'),
 ('northern', 'JJ'),
 ('Sumatra', 'NNP'),
 ('Philippine

In [30]:
nltk.ne_chunk(tag_elements).draw()

In [31]:
def extract_entities(chunk_tree):
    persons = []
    organizations = []
    locations = []
    
    for subtree in chunk_tree:
        if isinstance(subtree, nltk.Tree):
            if subtree.label() == 'PERSON':
                persons.append(' '.join([leaf[0] for leaf in subtree.leaves()]))
            elif subtree.label() == 'ORGANIZATION':
                organizations.append(' '.join([leaf[0] for leaf in subtree.leaves()]))
            elif subtree.label() == 'LOCATION':  # Geo-Political Entity
                locations.append(' '.join([leaf[0] for leaf in subtree.leaves()]))
    
    return persons, organizations, locations

# Usage
persons, organizations, locations = extract_entities(nltk.ne_chunk(tag_elements))
print("Persons:", persons)
print("Organizations:", organizations)
print("Locations:", locations)

Persons: ['Alaska', 'Peninsula Tonga', 'Mauritius', 'Edward Islands', 'Bouvet Island', 'Fiji', 'Okhotsk', 'Sumatra Philippine Islands', 'Islands Macquarie Island South Indian Ocean']
Organizations: ['Fiji', 'Loyalty Islands Kermadec Islands', 'Kermadec Islands Vanuatu', 'Kuril Islands']
Locations: ['South Sandwich Islands']


In [32]:
# List of words to remove
words_to_remove = ['Islands', 'Island', 'East', 'West', 'North', 'South',"Peninsula","Ocean"]

# Function to remove the unwanted words
def clean_list(word_list, words_to_remove):
    cleaned_list = []
    for item in word_list:
        # Split each phrase into words
        words = item.split()
        # Remove unwanted words
        cleaned_words = [word for word in words if word not in words_to_remove]
        # Rejoin the cleaned words back into a string
        cleaned_list.append(' '.join(cleaned_words))
    return cleaned_list

# Clean each list
cleaned_persons = clean_list(persons, words_to_remove)
cleaned_organizations = clean_list(organizations, words_to_remove)
cleaned_locations = clean_list(locations, words_to_remove)

# Print the cleaned lists
print("Cleaned Persons:", cleaned_persons)
print("Cleaned Organizations:", cleaned_organizations)
print("Cleaned Locations:", cleaned_locations)

Cleaned Persons: ['Alaska', 'Tonga', 'Mauritius', 'Edward', 'Bouvet', 'Fiji', 'Okhotsk', 'Sumatra Philippine', 'Macquarie Indian']
Cleaned Organizations: ['Fiji', 'Loyalty Kermadec', 'Kermadec Vanuatu', 'Kuril']
Cleaned Locations: ['Sandwich']


In [33]:
def split_list(li):
    list_1 = []
    for i in li:
        list_1.append(str(i).split())
    return list_1

In [34]:
cleaned_persons = split_list(cleaned_persons)
cleaned_organizations = split_list(cleaned_organizations)
cleaned_locations=split_list(cleaned_locations)

In [35]:
print("Cleaned Persons:", cleaned_persons)
print("Cleaned Organizations:", cleaned_organizations)
print("Cleaned Locations:", cleaned_locations)

Cleaned Persons: [['Alaska'], ['Tonga'], ['Mauritius'], ['Edward'], ['Bouvet'], ['Fiji'], ['Okhotsk'], ['Sumatra', 'Philippine'], ['Macquarie', 'Indian']]
Cleaned Organizations: [['Fiji'], ['Loyalty', 'Kermadec'], ['Kermadec', 'Vanuatu'], ['Kuril']]
Cleaned Locations: [['Sandwich']]


In [36]:
# Combine the cleaned lists into a single list of words
all_keywords = cleaned_persons + cleaned_organizations + cleaned_locations




In [37]:
df.isna().sum()

title          0
magnitude      0
date_time      0
cdi            0
mmi            0
alert        551
tsunami        0
sig            0
net            0
nst            0
dmin           0
gap            0
magType        0
depth          0
latitude       0
longitude      0
location       0
continent    713
country       51
dtype: int64

In [38]:
df["location"].unique()

array(['Sola, Vanuatu', 'Intipucá, El Salvador', 'Loncopué, Argentina',
       'Sand Point, Alaska', 'Alaska Peninsula',
       'Codrington, Antigua and Barbuda', 'Tonga', 'the Fiji Islands',
       'Panama-Colombia border region', 'the Loyalty Islands',
       'Hihifo, Tonga', 'Teluk Dalam, Indonesia',
       'Kermadec Islands, New Zealand', 'Tuban, Indonesia',
       'the Kamchatka Peninsula, Russia', 'New Guinea, Papua New Guinea',
       'San Antonio de los Cobres, Argentina', 'Jurm, Afghanistan',
       'Ecuador', 'Kermadec Islands region', 'Port-Olry, Vanuatu',
       'Kimbe, Papua New Guinea', 'Murghob, Tajikistan',
       'Ekinözü, Turkey', 'Nurdağı, Turkey', 'Central Turkey',
       'Santiago del Estero, Argentina', 'Tobelo, Indonesia',
       'Pulau Pulau Tanimbar, Indonesia', 'Vanuatu',
       'Malango, Solomon Islands', 'Bengkulu, Indonesia', 'Eastern, Fiji',
       'Neiafu, Tonga', 'Boca Chica, Panama', 'Aguililla, Mexico',
       'Yujing, Taiwan', 'Lugu, Taiwan', 'Isangel